In [1]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
import os, sys
from tqdm import tqdm

sys.path.insert(0, 'src')
from utils import remove_and_create_dir, ensure_dir
from preprocess_data import preprocess_csvs

/n/fs/ac-project/anaconda3/envs/dqn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Process a few data files

In [ ]:
data_root_dir =  os.path.join('data') # TODO: replace with your root if necessary
csv_file_names = ['fattree_rsim1.csv']
csv_paths = [os.path.join(data_root_dir, csv_file_name) for csv_file_name in csv_file_names]
save_dir = os.path.join('data', 'processed_data')

preprocess_csvs(
    csv_paths, 
    verbose=True,
    csv_save_dir=save_dir)

### Process a LOT of data files organized in directories

In [2]:
data_root_dir =  os.path.join('data') # TODO: replace with your root if necessary
# csv_file_names = ['rsim.csv', 'test.csv', 'train.csv']
sub_dirs = ['itr{}'.format(i) for i in range(1,6)]
csv_file_names = [
    'sch_FIFO-tgen_Poisson-n_flows_100-n_ports_4.csv',
    'sch_DDR-tgen_Poisson-n_flows_100-n_ports_4.csv',
    'sch_WFQ-tgen_Poisson-n_flows_100-n_ports_4.csv',
    'sch_SP-tgen_Poisson-n_flows_100-n_ports_4.csv',
    'sch_FIFO-tgen_OnOff-n_flows_100-n_ports_4.csv',
    'sch_FIFO-tgen_Poisson-n_flows_20-n_ports_4.csv',
    'sch_FIFO-tgen_Poisson-n_flows_60-n_ports_4.csv',
    'sch_FIFO-tgen_Poisson-n_flows_100-n_ports_8.csv',
    'sch_FIFO-tgen_Poisson-n_flows_100-n_ports_16.csv',
]
# print(sub_dirs)

for sub_dir in sub_dirs:
    csv_paths = []
    for csv_file_name in csv_file_names:
        csv_paths.append(os.path.join(data_root_dir, sub_dir, csv_file_name))

    # csv_paths = [os.path.join(data_root_dir, csv_file_name) for csv_file_name in csv_file_names]
    save_dir = os.path.join('data', 'processed_data', sub_dir)
    ensure_dir(save_dir)
    # print(len(csv_paths))
    preprocess_csvs(
        csv_paths, 
        verbose=False,
        csv_save_dir=save_dir)

Processing 1/9 csv: data/itr1/sch_FIFO-tgen_Poisson-n_flows_100-n_ports_4.csv


100%|███████████████████████████████████████████████████████████████████████████████████| 36/36 [00:37<00:00,  1.05s/it]


Saved processed csv to data/processed_data/itr1/sch_FIFO-tgen_Poisson-n_flows_100-n_ports_4_processed.csv

Processing 2/9 csv: data/itr1/sch_DDR-tgen_Poisson-n_flows_100-n_ports_4.csv


100%|███████████████████████████████████████████████████████████████████████████████████| 36/36 [00:38<00:00,  1.08s/it]


Saved processed csv to data/processed_data/itr1/sch_DDR-tgen_Poisson-n_flows_100-n_ports_4_processed.csv

Processing 3/9 csv: data/itr1/sch_WFQ-tgen_Poisson-n_flows_100-n_ports_4.csv


...

Saved processed csv to data/processed_data/itr2/sch_WFQ-tgen_Poisson-n_flows_100-n_ports_4_processed.csv

Processing 4/18 csv: data/itr1/sch_SP-tgen_Poisson-n_flows_100-n_ports_4.csv


100%|███████████████████████████████████████████████████████████████████████████████████| 36/36 [00:36<00:00,  1.01s/it]


Saved processed csv to data/processed_data/itr2/sch_SP-tgen_Poisson-n_flows_100-n_ports_4_processed.csv

Processing 5/18 csv: data/itr1/sch_FIFO-tgen_OnOff-n_flows_100-n_ports_4.csv


  6%|████▋                                                                               | 2/36 [01:28<24:59, 44.10s/it]


KeyboardInterrupt: 